In [1]:
%%capture
!pip install networkx
!pip install pandas

In [2]:
import json
import networkx as nx
import pandas as pd

# A. Temporal Graphs

In [3]:
def create_graph(graph_data: list, target_year: int, weighted: bool = False):
    graph = nx.Graph()
    for author1, author2, year in graph_data:
        if year != target_year: continue
        if graph.has_edge(author1, author2):
            graph[author1][author2]["weight"] = graph[author1][author2]["weight"] + 1
        else:
            graph.add_edge(author1, author2, weight=1)
    return graph
        

In [4]:
def load_json(file_path: str):
    with open(file_path, "r") as file:
        graph_data = json.load(file)
        return graph_data

In [5]:
graph_data = load_json("./tmp_dblp_coauthorship.json")

In [6]:
dblp2005 = create_graph(graph_data, 2005)
dblp2006 = create_graph(graph_data, 2006)
dblp2005w = create_graph(graph_data, 2005, weighted=True)

In [7]:
table = pd.DataFrame(columns=["Graph Name", "Node Count", "Edge Count"])
table.loc[len(table)] = ["dblp2005", dblp2005.number_of_nodes(), dblp2005.number_of_edges()]
table.loc[len(table)] = ["dblp2006", dblp2006.number_of_nodes(), dblp2006.number_of_edges()]
table.loc[len(table)] = ["dblp2005w", dblp2005w.number_of_nodes(), dblp2005w.number_of_edges()]

print(table)

  Graph Name  Node Count  Edge Count
0   dblp2005      180227      403026
1   dblp2006      201298      465988
2  dblp2005w      180227      403026


## Giant Connected Components

In [8]:
def giant_connected_component(graph):
    giant_cc_nodeset = max(nx.connected_components(graph), key=len)
    return graph.subgraph(giant_cc_nodeset)

In [9]:
dblp2005 = giant_connected_component(dblp2005)
dblp2006 = giant_connected_component(dblp2006)
dblp2005w = giant_connected_component(dblp2005w)

## Report

In [10]:
table = pd.DataFrame(columns=["Graph Name", "GCC Node Count", "GCC Edge Count"])
table.loc[len(table)] = ["dblp2005", dblp2005.number_of_nodes(), dblp2005.number_of_edges()]
table.loc[len(table)] = ["dblp2006", dblp2006.number_of_nodes(), dblp2006.number_of_edges()]
table.loc[len(table)] = ["dblp2005w", dblp2005w.number_of_nodes(), dblp2005w.number_of_edges()]

print(table)

  Graph Name  GCC Node Count  GCC Edge Count
0   dblp2005          106943          300043
1   dblp2006          123808          356968
2  dblp2005w          106943          300043


# B. Node and Edge Importance in Graphs

In [46]:
def pagerank(graph):
    pagerank = nx.pagerank(graph)
    return pagerank

def edge_betweenness(graph, k=10, weight=None, normalized=False):
    betweenness = nx.edge_betweenness_centrality(graph, k=k, weight=weight, normalized=normalized)
    return betweenness
    
def report_node_importance(graph_name, pagerank, top_n=50):
    node_importance_table = pd.DataFrame(columns=["Author name", "Pagerank score"])
    ranked_nodes = sorted(pagerank, reverse=True, key=lambda node: pagerank[node])
    for i in range(0,top_n):
        if i >= len(ranked_nodes): break
        node = ranked_nodes[i]
        rank = pagerank[node]
        node_importance_table.loc[len(node_importance_table)] = [node, rank]
    
    print(f"Graph: {graph_name}")
    print("Node importance:")
    print(node_importance_table)

def report_edge_importance(graph_name, betweenness, top_n=20):
    edge_importance_table = pd.DataFrame(columns=["Author 1", "Author 2", "Betweeness score"])
    ranked_edges = sorted(betweenness, reverse=True, key=lambda edge: betweenness[edge])
    for i in range(0, top_n):
        if i >= len(ranked_edges): break
        edge = ranked_edges[i]
        rank = betweenness[edge]
        edge_importance_table.loc[len(edge_importance_table)] = [edge[0], edge[1], rank]
        
    print(f"Graph: {graph_name}")
    print("Edge importance:")
    print(edge_importance_table)

## Report

In [29]:
dblp2005_pagerank = pagerank(dblp2005)
dblp2006_pagerank = pagerank(dblp2006)
dblp2005w_pagerank = pagerank(dblp2005w)

report_node_importance("dblp2005", dblp2005_pagerank)
report_node_importance("dblp2006", dblp2006_pagerank)
report_node_importance("dblp2005w", dblp2005w_pagerank)

Graph: dblp2005
Node importance:
          Author name  Pagerank score
0             Wen Gao        0.000175
1     Chin-Chen Chang        0.000152
2         Wei-Ying Ma        0.000135
3              Xin Li        0.000128
4        Licheng Jiao        0.000124
5    Francky Catthoor        0.000122
6     H. Vincent Poor        0.000119
7          Zhaohui Wu        0.000118
8   Hans-Peter Seidel        0.000117
9       Xianlong Hong        0.000114
10     Mario Piattini        0.000109
11          Wei Zhang        0.000108
12   Tharam S. Dillon        0.000108
13          Minglu Li        0.000108
14      Samuel Pierre        0.000104
15     Witold Pedrycz        0.000102
16      Guanrong Chen        0.000101
17            Wei Liu        0.000101
18            Hai Jin        0.000100
19     David J. Evans        0.000098
Graph: dblp2006
Node importance:
         Author name  Pagerank score
0            Wen Gao        0.000167
1    Chin-Chen Chang        0.000143
2            Qing Li     

In [ ]:
dblp2005_betweenness = edge_betweenness(dblp2005)
dblp2006_betweenness = edge_betweenness(dblp2006)
#dblp2005w_betweenness = edge_betweenness(dblp2005w, weight=weight)

report_edge_importance("dblp2005", dblp2005_betweenness)
report_edge_importance("dblp2006", dblp2006_betweenness)
#report_edge_importance("dblp2005w", dblp2005w_betweenness)

# C. Link Prediction in Graphs

## Pruning graphs

In [ ]:
def extract_subgraph_by_degree_count(graph, min_degree: int):
    nodes_to_keep = [node for node, degree in graph.degree if degree >= min_degree]
    return graph.subgraph(nodes_to_keep)

In [ ]:
dblp2005_core = extract_subgraph_by_degree_count(dblp2005, 3)
dblp2006_core = extract_subgraph_by_degree_count(dblp2006, 3)

## Prediction

In [ ]:
def friends_of_friends_connections(graph):
    fof_connections = []
    visited = set()
    for node in graph.nodes():
        fof_set = set()
        for friend in graph[node]:
            if friend == node: continue
            for fof in graph[friend]:
                if fof == node or fof in visited:
                    continue
                fof_set.add(fof)
        
        for fof in fof_set:
            fof_connections.append((node, fof))
                
        visited.add(node)
    return fof_connections
                    
    
def graph_edge_difference(minuend_graph, subtrahend_graph):
    edge_difference = []
    for edge in minuend_graph.edges():
        if subtrahend_graph.has_edge(*edge):
            continue
        edge_difference.append(edge)
    return edge_difference

In [ ]:
candidate_edges = friends_of_friends_connections(dblp2005_core)
ground_truth_edges = graph_edge_difference(dblp2006_core, dblp2005_core)

## Precision

In [ ]:
def precision_at_k(k_values, predictions, ground_truth, include_max_k=False):
    if type(k_values)==int:
        k_values = [k_values]
    ground_truth = set(ground_truth)
    precision_results = dict()
    
    # `predictions` is of the form (u,v,s) where (u,v) is the edge, and s is the score for the edge for the specific link prediction algorithm that produced those predictions
    sorted_predictions = sorted(predictions, key=lambda pred: pred[2])
    predictions = [(u, v) for (u,v,s) in sorted_predictions]
    
    k = 0
    correct_pred_count = 0
    for pred in predictions:
        k += 1
        if pred in ground_truth:
            correct_pred_count += 1
        
        if k in k_values:
            precision_results[k] = correct_pred_count/k
    
    if include_max_k and k>0 and k not in precision_results:
        precision_results[k] = correct_pred_count/k
    
    return precision_results

def precision_report(predictions, ground_truth):
    fixed_k_vals = [10, 20, 50, 100]
    precision_results = precision_at_k(fixed_k_vals, predictions, ground_truth, include_max_k=True)
    k_vals = sorted(precision_results.keys())
    if len(precision_results)==0:
        return pd.DataFrame()
    
    precision_report = pd.DataFrame(columns=[f"P@{k}" for k in k_vals])
    precision_report.loc[len(precision_report)] = [precision_results[k] for k in k_vals]
    
    return precision_report
        
    

In [ ]:
print("P@k values for Jaccard Coefficient")
jc_pred = nx.jaccard_coefficient(dblp2005_core, candidate_edges)
print(precision_report(jc_pred, ground_truth_edges))

In [ ]:
print("P@k values for Preferential Attachment")
pa_pred = nx.preferential_attachment(dblp2005_core, candidate_edges)
print(precision_report(pa_pred, ground_truth_edges))

In [ ]:
print("P@k values for Adamic/Adar")
aa_pred = nx.adamic_adar_index(dblp2005_core, candidate_edges)
print(precision_report(aa_pred, ground_truth_edges))

In [ ]:
print("P@k values for Common Neighbors")
cn_pred = nx.cn_soundarajan_hopcroft(dblp2005_core, candidate_edges, community=None)
print(precision_report("Common Neighbors", cn_pred, ground_truth_edges))